## Simple benchmark using XGB

Benchamrk with XGB item_cnt_month clipped between o to 20, validation on month before prediction (33)
Final model is built using months upto 32 and final prediction is done on test data using that model.

In [1]:
import os
import sys

import pandas as pd
import xgboost as xgb

sys.path.insert(0, os.path.abspath('/home/jupyter/kaggle/predict_future_sales/src/'))

import common.com_util as util
import config.constants as constants
import cv.cv_util as cv
import munging.process_data as process_data
import modeling.train_util as model

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
logger = util.get_logger('sub_2')

train_df = pd.read_feather('/home/jupyter/kaggle/predict_future_sales/data/processed/train_all_merged.feather')
test_df = pd.read_feather('/home/jupyter/kaggle/predict_future_sales/data/processed/test_all_merged.feather')

train_features = ['shop_id', 'item_id', 'date_block_num', 'item_category_id', 'item_cnt_month']
train_df = train_df[train_features]

test_features = ['ID', 'shop_id', 'item_id', 'date_block_num', 'item_category_id']
test_df = test_df[test_features]

sample_submission = pd.read_feather('/home/jupyter/kaggle/predict_future_sales/data/processed/submission_processed.feather')

test = test_df.drop(['ID'], axis='columns')

# Clip the item_cnt_month within 20
train_df.item_cnt_month.clip(lower=0, upper=20, inplace=True)

/home/jupyter/anaconda3/envs/py37/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [4]:
TARGET = 'item_cnt_month'
ID = 'ID'
SEED = 42

training_months = [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
holdout_months = [33]

training, validation = cv.get_data_splits_by_date_block(logger, train_df, 
                                                        train_months=training_months, 
                                                        validation_months=holdout_months)

[INFO]2020-09-09 06:24:02,158:sub_2:Splitting the data into train and holdout based on months...
[INFO]2020-09-09 06:24:02,159:sub_2:Training months [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
[INFO]2020-09-09 06:24:02,160:sub_2:Validation months [33]
[INFO]2020-09-09 06:24:03,058:sub_2:Shape of the training data (10675678, 5) 
[INFO]2020-09-09 06:24:03,059:sub_2:Shape of the validation data (238172, 5)


In [5]:
xgb_params = {
                # Learning task parameters
                'objective': 'reg:squarederror',
                'eval_metric': 'rmse',
                'seed': SEED,

                # Type of the booster
                'booster': 'gbtree',
                
                # parameters for tree booster
                'learning_rate': 0.3,
                'max_depth': 6,
                'max_leaves': 0,
                'max_bin': 256,
                'early_stopping_rounds' : 100,

                # General parameters
                #'verbosity': 2, #info
                }

In [6]:
predictors = ['shop_id', 'item_id', 'date_block_num', 'item_category_id']

In [7]:
training.shape

(10675678, 5)

In [8]:
validation.shape

(238172, 5)

In [9]:
training_merged = pd.concat([training, validation])
training_merged.shape

(10913850, 5)

In [10]:
test.head()

,shop_id,item_id,date_block_num,item_category_id
0,5,5037,34,19
1,5,5320,34,55
2,5,5233,34,19
3,5,5232,34,23
4,5,5268,34,20


In [12]:
bst, best_iteration, valid_score, prediction = model.xgb_train_validate_on_holdout(
        logger=logger, training=training, validation=validation, 
        predictors=predictors, target=TARGET, params=xgb_params, test_X=test, 
        n_estimators=10000, early_stopping_rounds=100, verbose_eval=100
    )

[INFO]2020-09-09 06:28:22,054:sub_2:Training using XGBoost and validating on holdout
[INFO]2020-09-09 06:28:22,132:sub_2:Shape of train_X, train_Y, validation_X, validation_Y: (10675678, 4), (10675678,), (238172, 4), (238172,)
[06:28:26] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[0]	train-rmse:1.20852	valid_data-rmse:1.1299
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 100 rounds.
[100]	train-rmse:1.09585	valid_data-rmse:1.01315
[200]	train-rmse:1.08463	valid_data-rmse:1.01309
Stopping. Best iteration:
[157]	train-rmse:1.08807	valid_data-rmse:1.01163

[INFO]2020-09-09 06:46:10,861:sub_2:Validation Score 1.0116379261016846
[INFO]2020-09-09 06:46:10,865:sub_2:Best Iteration 1

/home/jupyter/anaconda3/envs/py37/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/jupyter/anaconda3/envs/py37/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[06:46:15] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[INFO]2020-09-09 06:57:22,240:sub_2:Predicting on test data: (214200, 4)


In [14]:
bst, best_iteration, valid_score

(<xgboost.core.Booster at 0x7f103aefc090>, 158, 1.0116379)

In [15]:
prediction

array([0.38830838, 0.24504653, 0.13283464, ..., 0.09535095, 0.09556562,
       0.20091727], dtype=float32)

In [16]:
test_df.shape

(214200, 5)

In [17]:
sample_submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [18]:
submission = pd.DataFrame({'ID': test_df.ID, 'item_cnt_month': prediction})

In [19]:
submission.head()

,ID,item_cnt_month
0,0,0.388308
1,1,0.245047
2,2,0.132835
3,3,-0.011641
4,4,0.167597
